In [1]:
import json

from enbios.generic.mermaid2hierarchy import convert_mermaid_lines

In [2]:
mermaid1 = """graph BT
b1-->a
b2-->a"""

In [3]:
hierarchy = convert_mermaid_lines(mermaid1)
hierarchy

{'name': 'a',
 'aggregator': None,
 'config': None,
 'children': [{'name': 'b1', 'adapter': None, 'config': None},
  {'name': 'b2', 'adapter': None, 'config': None}]}

In [4]:
print(json.dumps(hierarchy, indent=2, ensure_ascii=False))

{
  "name": "a",
  "aggregator": null,
  "config": null,
  "children": [
    {
      "name": "b1",
      "adapter": null,
      "config": null
    },
    {
      "name": "b2",
      "adapter": null,
      "config": null
    }
  ]
}


In [5]:
# Specifying adapters and aggregators wtih ':::' (which can be done once per node)

In [6]:
mermaid2 = """graph BT
b1-->a:::sum
b2:::bw-->a"""

hierarchy = convert_mermaid_lines(mermaid2)
hierarchy

{'name': 'a',
 'aggregator': 'sum',
 'config': None,
 'children': [{'name': 'b1', 'adapter': None, 'config': None},
  {'name': 'b2', 'adapter': 'bw', 'config': None}]}

In [7]:
mermaid2b = """
graph BT
    b1 --> a:::sum
    b2 -- bw --> a
"""

print(json.dumps(convert_mermaid_lines(mermaid2), indent=2, ensure_ascii=False))

{
  "name": "a",
  "aggregator": "sum",
  "config": null,
  "children": [
    {
      "name": "b1",
      "adapter": null,
      "config": null
    },
    {
      "name": "b2",
      "adapter": "bw",
      "config": null
    }
  ]
}


In [8]:
mermaid2 = """graph BT
a:::sum
b1:::bw
b2:::bw
b1-->a
b2-->a"""

print(json.dumps(convert_mermaid_lines(mermaid2), indent=2, ensure_ascii=False))

{
  "name": "a",
  "aggregator": "sum",
  "config": null,
  "children": [
    {
      "name": "b1",
      "adapter": "bw",
      "config": null
    },
    {
      "name": "b2",
      "adapter": "bw",
      "config": null
    }
  ]
}


In [9]:
mermaid3 = """graph BT
a:::sum
b1:::bw,code:xxx1
b2:::bw,code:xxx2

b1-->a
b2-->a"""

print(json.dumps(convert_mermaid_lines(mermaid3), indent=2, ensure_ascii=False))

{
  "name": "a",
  "aggregator": "sum",
  "config": null,
  "children": [
    {
      "name": "b1",
      "adapter": "bw",
      "config": {
        "code": "xxx1"
      }
    },
    {
      "name": "b2",
      "adapter": "bw",
      "config": {
        "code": "xxx2"
      }
    }
  ]
}


## How to read a mermaid file convert it and add it to some experiment configuration

In [1]:
from enbios.generic.mermaid2hierarchy import convert_mermaid_file

hierarchy = convert_mermaid_file("data/mermaid/simple_mermaid.mmd")

In [3]:
from enbios.base.experiment import Experiment

exp_config = {
    "adapters": [
        {
            "adapter_name": "brightway-adapter",
            "config": {"bw_project": "ecoinvent_391"},
            "methods": {
                "GWP1000": (
                    "ReCiPe 2016 v1.03, midpoint (H)",
                    "climate change",
                    "global warming potential (GWP1000)",
                ),
                "LOP": (
                    "ReCiPe 2016 v1.03, midpoint (E)",
                    "land use",
                    "agricultural land occupation (LOP)",
                ),
                "WCP": (
                    "ReCiPe 2016 v1.03, midpoint (E)",
                    "water use",
                    "water consumption potential (WCP)",
                ),
            },
        }
    ],
    "hierarchy": hierarchy,
}

exp = Experiment(exp_config)

In [4]:
exp.run()

2024-02-21 14:23:53,782 - ............demos.enbios.base - INFO - Running scenario 'default scenario'


{'default scenario': {'name': 'root',
  'results': {'GWP1000': {'unit': 'kg CO2-Eq',
    'magnitude': 0.032264059590470524},
   'LOP': {'unit': 'm2*a crop-Eq', 'magnitude': 0.001447083969231639},
   'WCP': {'unit': 'cubic meter', 'magnitude': 0.00032440108814901574}},
  'output': {'unit': 'kilowatt_hour', 'magnitude': 2.0},
  'children': [{'name': 'b1',
    'results': {'GWP1000': {'unit': 'kg CO2-Eq',
      'magnitude': 0.015243611332791748},
     'LOP': {'unit': 'm2*a crop-Eq', 'magnitude': 0.0010661197743596256},
     'WCP': {'unit': 'cubic meter', 'magnitude': 0.000136359937516592}},
    'output': {'unit': 'kilowatt_hour', 'magnitude': 1.0}},
   {'name': 'b2',
    'results': {'GWP1000': {'unit': 'kg CO2-Eq',
      'magnitude': 0.017020448257678778},
     'LOP': {'unit': 'm2*a crop-Eq', 'magnitude': 0.00038096419487201346},
     'WCP': {'unit': 'cubic meter', 'magnitude': 0.00018804115063242375}},
    'output': {'unit': 'kilowatt_hour', 'magnitude': 1.0}}]}}